In [1]:
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.functions import col , current_timestamp , year , month , day , hour
import time
import datetime

In [2]:
spark = SparkSession.builder.appName("Save output into HDFS") \
                    .config('spark.driver.memory' , '4g') \
                    .getOrCreate()

Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
25/11/01 01:41:00 WARN Utils: Your hostname, Admin-PC, resolves to a loopback address: 127.0.1.1; using 10.255.255.254 instead (on interface lo)
25/11/01 01:41:00 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/11/01 01:41:01 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
25/11/01 01:41:02 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [15]:
def save_hdfs(df , name) :
    output = "hdfs://localhost:9000/Week3/" + name + "/"
    df = df.withColumn('created_date' , current_timestamp()) \
            .withColumn('updated_date' , current_timestamp()) 
    df_partitions = df.withColumn('year_created' , year(col('created_date'))) \
                        .withColumn('month' , month(col('created_date'))) \
                        .withColumn('day' , day(col('created_date'))) \
                        .withColumn('hour' , hour(col('created_date'))) 

    df_partitions.write.format('parquet').partitionBy('year_created' , 'month' , 'day' , 'hour').mode('overwrite').save(output)

In [16]:
list = ['animes_best_rated' , 'genres_user' , 'type_per_year']

for name in list :
    df = spark.read.parquet(f"../output/parquet/{name}/{name}.parquet")
    save_hdfs(df , name)
